# Social Security file Validator

## Imports


In [ ]:
import os
import subprocess
import ipywidgets as widgets
from IPython.display import FileLink, display
from SPARQLWrapper import SPARQLWrapper, TURTLE

## Select declaration type and quarter

Valid declaration type: "dmfa", "dmfareq"<br>Valid quarter: 20223, 20222, 20221

In [ ]:
declTypeWidget = widgets.Dropdown(
    options=['dmfa', 'dmfareq'],
    value='dmfa',
)

quarterWidget = widgets.Dropdown(
    options=['20223', '20222', '20221'],
    value='20223',
)
display(declTypeWidget)
display(quarterWidget)

In [ ]:
declaration = declTypeWidget.value
quarter = quarterWidget.value

## Input file to validate

Filepath

In [ ]:
filepath = input() # ex: valid_dmfa.ttl, invalid_dmfa.ttl

if os.path.isfile(filepath):
    print("File exists")    
else:
    print("File does not exist")

File Upload

In [ ]:
uploader = widgets.FileUpload(multiple=False)
display(uploader)

In [ ]:
uploadsDir = "uploads"

if not os.path.exists(uploadsDir):
    os.makedirs(uploadsDir)

if uploader.value:
    fileDict = next(iter(uploader.value))
    print(fileDict)
    filename = fileDict["name"]
    fileContent = fileDict["content"]

    filepath = os.path.join(uploadsDir, filename)
    with open(filepath, "wb") as file:
        file.write(fileContent)

## Input output file name

In [ ]:
outfilename = input() #ex: validation_report.ttl

## Input triple store endpoint

In [ ]:
endpoint = "http://localhost:3030/socialsecurity"

## Validation

In [ ]:
# SPARQL endpoint setup
from SPARQLWrapper import SPARQLWrapper, TURTLE
sparql = SPARQLWrapper(endpoint)
sparql.setReturnFormat(TURTLE)

Creating data graph


In [ ]:
# Copy input file
import shutil
datagraphFilepath = filepath.split(".")[0] + "_datagraph_tmp.ttl"
shutil.copy(filepath, datagraphFilepath)

# Fetch and append annexes
FETCH_ANNEXES_QUERY = """
CONSTRUCT{
	?s ?p ?o.
}
FROM <http://kg.socialsecurity.be/resource/annex2/>
FROM <http://kg.socialsecurity.be/resource/annex4/>
FROM <http://kg.socialsecurity.be/resource/annex6/>
FROM <http://kg.socialsecurity.be/resource/annex7/>
FROM <http://kg.socialsecurity.be/resource/annex8/>
FROM <http://kg.socialsecurity.be/resource/annex9/>
FROM <http://kg.socialsecurity.be/resource/annex10/>
FROM <http://kg.socialsecurity.be/resource/annex11/>
FROM <http://kg.socialsecurity.be/resource/annex21/>
FROM <http://kg.socialsecurity.be/resource/annex27/>
FROM <http://kg.socialsecurity.be/resource/annex28/>
FROM <http://kg.socialsecurity.be/resource/annex31/>
FROM <http://kg.socialsecurity.be/resource/annex35/>
FROM <http://kg.socialsecurity.be/resource/annex42/>
FROM <http://kg.socialsecurity.be/resource/annex43/>
FROM <http://kg.socialsecurity.be/resource/annex44/>
FROM <http://kg.socialsecurity.be/resource/annex45/>
FROM <http://kg.socialsecurity.be/resource/annex46/>
FROM <http://kg.socialsecurity.be/resource/dmfas02/>
WHERE{
	?s ?p ?o.
}
"""

sparql.setQuery(FETCH_ANNEXES_QUERY)
response = sparql.queryAndConvert().decode("utf-8")

datagraph = open(datagraphFilepath, mode="a", encoding="utf-8")
datagraph.write("\n" + response)
datagraph.close()

Creating shape graph


In [ ]:
FETCH_SHAPES_QUERY = """
CONSTRUCT{{
	?s ?p ?o.
}}
FROM <http://kg.socialsecurity.be/resource/shapes/common/{0}/>
FROM <http://kg.socialsecurity.be/resource/shapes/{1}/{0}/>
WHERE{{
	?s ?p ?o.
}}
"""

sparql.setQuery(FETCH_SHAPES_QUERY.format(quarter, declaration))
response = sparql.queryAndConvert().decode("utf-8")

shapegraphFilename = "shapes_tmp.ttl"
shapegraph = open(shapegraphFilename, mode="w", encoding="utf-8")
shapegraph.write(response)
shapegraph.close()

Validate data graph against shape graph with TopBraid


In [ ]:
downloadsDir = "downloads"

if not os.path.exists(downloadsDir):
    os.makedirs(downloadsDir)

outfilepath = os.path.join(downloadsDir, outfilename)

command = "shaclvalidate.sh -datafile {} -shapesfile {}  > {}".format(datagraphFilepath, shapegraphFilename, outfilepath)
subprocess.run(command, shell=True)
download_link = FileLink(outfilepath, result_html_prefix="Click here to download: ")
display(download_link)

## Clean up

In [ ]:
os.remove(datagraphFilepath)
os.remove(shapegraphFilename)